source:

NCBI viral genomes resource.
Brister JR, Ako-Adjei D, Bao Y, Blinkova O. Nucleic Acids Res. 2015 Jan;43(Database issue):D571-7. doi: 10.1093/nar/gku1207. Epub 2014 Nov 26.

In [1]:
import pandas as pd
import bs4 as bs
import requests
import numpy as np
import progressbar
import os
import csv
from Bio import Entrez
import re
import time
import sys

## Data Acquisition

In [2]:
# Create a BeautifulSoup object for the main page
url = 'https://www.ncbi.nlm.nih.gov/genomes/GenomesGroup.cgi'
result = requests.get(url)
main_soup = bs.BeautifulSoup(result.content, 'html.parser')

In [3]:
#Parse the main_soup for the table of viruses
table = main_soup.find('table', {'class': 'tblTxt'})

In [4]:
# This table contains some subtables, which are included within single cells, called 'minitable' below
# The rows in the minitables are formatted differently, but contain the same info that I want, so this function 
# will convert them into the same format as the rest. A list comprehension didn't work because the 'minirows' have 
# variable lengths
def edit_minirow(row): 
    n = len(row)
    #assuming edited rows should have structure =[row[0],row[2],'None','None', row[1],row[3], row[4]]
    if n == 1:
        new_minirow = [row[0], 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']
    elif n == 2:
        new_minirow = [row[0], 'None', 'None', 'None', row[1], 'None', 'None', 'None', 'None', 'None', 'None']
    elif n == 3:
        new_minirow = [row[0], row[2], 'None', 'None', row[1], 'None', 'None', 'None', 'None', 'None', 'None']
    elif n == 4:
        new_minirow = [row[0], row[2], 'None', 'None', row[1], row[3], 'None', 'None', 'None', 'None', 'None']
    elif n == 5:
        new_minirow = [row[0], row[2], 'None', 'None', row[1], row[3], row[4], 'None', 'None', 'None', 'None']
    else:
        print('error:', row)
    return(new_minirow)

# Initialize the list of classifications
class_list = ['Deltavirus', 'Genomoviridae', 'Retro-transcribing viruses', 'Satellites', 
'Virus families not assigned to an order', 'Virus-associated RNAs', 'dsDNA viruses, no RNA stage', 'dsRNA viruses',
'environmental samples', 'ssDNA viruses', 'ssRNA viruses', 'unassigned viruses', 'unclassified RNA viruses',
'unclassified archaeal viruses', 'unclassified bacterial viruses', 'unclassified virophages', 
'unclassified viruses']

In [5]:
# Create an empty list of lists
viral_genome_array = []

# Create a list of rows that will ensure that for every row in a minitable, that row isn't copied again when the 
#  main table finds those rows. The alternative would be to not include a separate minitable code, but since the 
#  minitables are within cells, those cells' text is added to the table
minirow_master_list = []

# This is in order to add the previous non-minitable row to each minitable row, as it contains relevant info
head_row = []

# This variable will hold the subheader of the previous lighter colored column, which will contain extra info for the 
# light columns beneath it
sub_head = ''
sub_head_list = []

# Go through all of the rows in the table 
for row in table.find_all('tr'):
    this_row = []
    # Go through each cell in each row
    for cell in row.find_all('td'):
        
        # If the cell contains a table (the pulldown rows on the main page) reformat those rows like the normal rows
        try:
            minitable = cell.find('table')
            for minirow in minitable.find_all('tr'):
                this_minirow = []
                for minicell in minirow.find_all('td'):
                    if 'NC_' in minicell.text:
                        this_minirow.append(minicell.find('a').attrs['href'])
                    else:
                        this_minirow.append(minicell.text.replace('\n','').replace('\xa0',''))
                minirow_master_list.append(this_minirow)
                edited_minirow = edit_minirow(this_minirow)
                edited_minirow.append(head_row)
                edited_minirow[0] = edited_minirow[0] + ' segment '
                viral_genome_array.append(edited_minirow)
                
        # If there isn't a table, grab the href link for cells with a 'NC_', otherwise grab the text
        except AttributeError:
            if 'NC_' in cell.text:
                this_row.append(cell.find('a').attrs['href'])
            else:
                this_row.append(cell.text.replace('\n','').replace('\xa0',''))
    
    # For the lighter rows, which share a subheader, either grab the subheader or append the current subheader in 
    # the 12th position (index 11)
    try:
        if row.attrs['bgcolor'] == '#F8F8F8':
            if len(this_row) == 1:
                # Classifications also have this lighter color, but I will to treat them differently to add them to 
                #  every row
                if this_row[0] not in class_list:
                    sub_head = this_row[0]
                    sub_head_list.append(this_row)
            else:
                while len(this_row) < 11:
                    this_row.append('None')
                this_row.append(sub_head)
    except:
        pass
    #
    
    # don't take rows that have already been taken down and reformatteed, and rows that include the text 
    # of a whole minitable (which have the name repeated)
    if this_row not in minirow_master_list and this_row[0] not in this_row[1:]: 
        head_row = this_row 
        viral_genome_array.append(this_row)
    
print('done')

done


In [6]:
# Add a classification and extra info feature in the first row (these will become dataframe column headers)
viral_genome_array[0].append('Classification')
viral_genome_array[0].append('Extra Info')

# If a row contains only a classification, everything below that row falls into that category until the next 
# classification comes up
classification = 'None'
for row in viral_genome_array[1:]:
    if row[0] in class_list:
        classification = row[0]
        viral_genome_array.remove(row)
    else:
        while len(row) < 11:
            row.append('None')
        row[10] = classification

In [7]:
# Gather a list of the head_rows that ended up in the extra info columns
head_row_list = []
for row in viral_genome_array[1:]:
    if len(row) == 12:
        head = row[11]
        if head != 'None' and head not in head_row_list:
            head_row_list.append(head)

# now that I have a list of heads, go through the array again and remove rows that match those head_rows, as well
#  as subheaders
for row in viral_genome_array[1:]:
    if row in head_row_list or row in sub_head_list:
        viral_genome_array.remove(row)        

In [8]:
# Extract the info in 'Extra Info' lists 
# For each row in the array, if the row ends in a list, replace missing values in that row with the corresponding 
#  value in the list
for row in viral_genome_array:
    if type(row[-1]) == list:
        extra_list = row[-1]
        for i in range(len(row)):
            if i not in [3,4,5,6]:
                if row[i] == 'None' or row[i] == '' or row[i] == '-':
                    row[i] = extra_list[i]

In [9]:
# Number all of the segments
segment_counts = {}

for row in viral_genome_array:
    name = row[0]
    if 'segment' in name:
        if name not in segment_counts:
            segment_counts[name] = 1
            row[0] += '1'
        else:
            segment_counts[name] += 1
            row[0] += str(segment_counts[name])

In [10]:
viral_genomes = pd.DataFrame(viral_genome_array[2:], columns = viral_genome_array[0])#, columns = viral_genome_array[0])
viral_genomes.head()

,Genome,Accession,Source information,Segm,Length,Protein,Neighbors,Host,Created,Updated,Classification,Extra Info
0,Hepatitis delta virus,/nuccore/13277517,,-,1682nt,2,320,"vertebrates, human",04/29/1993,02/10/2015,Deltavirus,None
1,Badger associated gemykibivirus 1,/nuccore/807743872,strain:588t,-,2112nt,2,-,vertebrates,04/16/2015,11/24/2015,Genomoviridae,None
2,Bemisia-associated genomovirus AdDF,/nuccore/1211677462,isolate:AdDF,-,2199nt,2,-,invertebrates,06/27/2017,07/19/2017,Genomoviridae,None
3,Bemisia-associated genomovirus AdO,/nuccore/1211677465,isolate:AdO,-,2211nt,2,-,invertebrates,06/27/2017,07/19/2017,Genomoviridae,None
4,Bemisia-associated genomovirus NfO,/nuccore/1211677468,isolate:NfO,-,2231nt,2,-,invertebrates,06/27/2017,07/19/2017,Genomoviridae,None


------------------------------------------------------------------------------------------------------------------
##### This cell gets the full html 'soup' from each virus' page, which includes the genome. My intention is to only run this once since it will take a while, and save the result into a csv file in this directory. 
    length = len(viral_genomes)
    
    soup_dict = {}

###### Using a progress bar, go through the dataframe, getting a soup for each genome and saving the result to a dict
    with progressbar.ProgressBar(max_value=length) as bar:
        for i in range(length):
    
            name = viral_genomes['Genome'][i]
        
            slug = viral_genomes['Accession'][i]
        
            result = requests.get('https://www.ncbi.nlm.nih.gov{}'.format(slug))
        
            soup = bs.BeautifulSoup(result.content, 'html.parser')
        
            soup_dict[name] = soup
        
            bar.update(i)
        
    os.system('say "download complete"')

###### Write the dict to a csv: "virus_soups.csv"
    w = csv.writer(open("virus_soups.csv", "w"))
    for key, val in soup_dict.items():
        w.writerow([key, val])
----------------------------------------------------------------------------------------------------------

In [11]:
# Load in the dataframe including the BeautifulSoups
meta = pd.read_csv('virus_soups.csv', header = None, names = ['Name', 'BeautifulSoup'])
meta['BeautifulSoup'] = meta['BeautifulSoup'].apply(lambda x: bs.BeautifulSoup(x, 'html.parser'))
meta.head()

,Name,BeautifulSoup
0,Hepatitis delta virus,"<?xml version=""1.0"" encoding=""utf-8""?> <!DOCTY..."
1,Badger associated gemykibivirus 1,"<?xml version=""1.0"" encoding=""utf-8""?> <!DOCTY..."
2,Bemisia-associated genomovirus AdDF,"<?xml version=""1.0"" encoding=""utf-8""?> <!DOCTY..."
3,Bemisia-associated genomovirus AdO,"<?xml version=""1.0"" encoding=""utf-8""?> <!DOCTY..."
4,Bemisia-associated genomovirus NfO,"<?xml version=""1.0"" encoding=""utf-8""?> <!DOCTY..."


In [12]:
# This function returns the id that I need for each virus to access it in the NCBI Entrez database 
def get_db_id (soup):
    try:
        line = soup.find('p', {'class': 'itemid'}).text
        itemid = (line[25:])
        return(itemid)
    except AttributeError:
        return(np.nan)

# Add the sequence type to the main DataFrame
def get_seq_type(soup):
    try:
        head = soup.find('div', {"class":"rprtheader"})
        line = head.find('h1')
        title = (line.text)
        sequence_type = re.search(',[a-z,0-9, ]+', title)[0]
        return(sequence_type[2:])
    except AttributeError:
        return('error')
    except TypeError:
        return('error')    
    
meta['ItemId'] = meta['BeautifulSoup'].apply(get_db_id)
viral_genomes['SequenceType'] = meta['BeautifulSoup'].apply(get_seq_type)

In [13]:
# Check to make sure we have an itemid for each virus/segment
meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9461 entries, 0 to 9460
Data columns (total 3 columns):
Name             9461 non-null object
BeautifulSoup    9461 non-null object
ItemId           9459 non-null object
dtypes: object(3)
memory usage: 221.8+ KB


In [14]:
# identify missing ids (np.nan is a float type)
for i in range(len(meta['ItemId'])):
    string = meta['ItemId'][i]
    if type(string) == float:
        print(i)

4240
6110


In [15]:
#manually enter ids for 4240, 6110

meta['ItemId'][4240] = 'NC_032149.2'
meta['ItemId'][6110] = 'NC_025484.1'

In [16]:
viral_genomes['SequenceType'].value_counts()

complete genome                                                                                                                                  5532
complete sequence                                                                                                                                1683
complete cds                                                                                                                                     1409
error                                                                                                                                             201
hypothetical protein 2, and hypothetical protein 3 genes, complete cds                                                                             93
                                                                                                                                                   63
segment                                                                                             

###### As with scraping the html for each virus above, I will attempt to only extract the report for each virus once, 
###### I will save this to the meta df and thereafter I'll be importing that and not running this cell. 

    meta['Report'] = 'None'
    length = len(meta['ItemId'])

    with progressbar.ProgressBar(max_value=length) as bar:
        for i in range(length):     #starting at 4235 because it stopped at 4240
            try:
                Entrez.email = 'qdupupet@umass.edu'
                handle = Entrez.efetch(db="nuccore", id=meta['ItemId'][i], rettype="gb", retmode="text")
                report = handle.read()
                meta['Report'][i] = report
            except AttributeErrror:
                pass
            time.sleep(1)
            bar.update(i)
        
    meta.to_csv('virus_meta.csv', index = False)

##### I didn't get all the reports, so I'll try getting the rest with a slightly different method

##### just until I have the final version
##### Read in meta csv with reports 0-4240 already in
    chunksize = 1000
    TextFileReader = pd.read_csv('virus_meta.csv', chunksize=chunksize, iterator=True)
    meta = pd.concat(TextFileReader, ignore_index=True)

    meta.head()

##### The extra reports will be loaded into a separate csv (writing to meta took too long) and the resuslts will be added to 'meta'
    reports = []
    length = len(meta['ItemId'])

    with progressbar.ProgressBar(max_value=length) as bar:
        for i in range(4235, length):
            line = [i]
            try:
                Entrez.email = 'qdupupet@umass.edu'
                handle = Entrez.efetch(db="nuccore", id=meta['ItemId'][i], rettype="gb", retmode="text")
                report = handle.read()
                line.append(report)
            except AttributeErrror:
                pass
            reports.append(line)
            time.sleep(1)
            bar.update(i)

    reports = pd.DataFrame(reports)
    reports.to_csv('virus_extra_reports.csv')

##### Add the batch of reports to the subset of the meta csv that they blong to 
    extra_report_dict = {}
    with open('virus_extra_reports.csv', 'r') as csvfile:
        report_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in report_reader:
            extra_report_dict[row[1]] = row[2]

    submeta = meta[:][4236:7093]

    submeta['Report'] = submeta['Unnamed: 0'].apply(lambda x: extra_report_dict[str(x)])

    submeta.tail()

    meta.to_csv('virus_meta.csv', index = False)

In [83]:
# read in the meta with reports 0-7092 already in 
chunksize = 1000
TextFileReader = pd.read_csv('virus_meta.csv', chunksize=chunksize, iterator=True)
meta = pd.concat(TextFileReader, ignore_index=True)

meta[7090:7095]

,Unnamed: 0,Name,BeautifulSoup,ItemId,Report
0,0,Hepatitis delta virus,"<?xml version=""1.0"" encoding=""utf-8""?>\n<!DOCT...",NC_001653.2,LOCUS NC_001653 1682 bp ss...
1,1,Badger associated gemykibivirus 1,"<?xml version=""1.0"" encoding=""utf-8""?>\n<!DOCT...",NC_026806.1,LOCUS NC_026806 2112 bp ...
2,2,Bemisia-associated genomovirus AdDF,"<?xml version=""1.0"" encoding=""utf-8""?>\n<!DOCT...",NC_035137.1,LOCUS NC_035137 2199 bp ...
3,3,Bemisia-associated genomovirus AdO,"<?xml version=""1.0"" encoding=""utf-8""?>\n<!DOCT...",NC_035138.1,LOCUS NC_035138 2211 bp ...
4,4,Bemisia-associated genomovirus NfO,"<?xml version=""1.0"" encoding=""utf-8""?>\n<!DOCT...",NC_035139.1,LOCUS NC_035139 2231 bp ...


##### The extra reports will be loaded into a separate csv (writing to meta took too long) and the resuslts will be added to 'meta'
    reports2 = []
    length = len(meta['ItemId'])

    with progressbar.ProgressBar(max_value=length) as bar:
        for i in range(7090, length):
            line = [i]
            try:
                Entrez.email = 'qdupupet@umass.edu'
                handle = Entrez.efetch(db="nuccore", id=meta['ItemId'][i], rettype="gb", retmode="text")
                report = handle.read()
                line.append(report)
            except AttributeErrror:
                pass
            reports2.append(line)
            time.sleep(1)
            bar.update(i)

    reports2 = pd.DataFrame(reports2)
    reports2.to_csv('virus_extra_reports2.csv', index = False)

##### Write the last batch of reports to a dictionary and enter them into the DataFrame subset starting at index 7090
    csv.field_size_limit(sys.maxsize)
    
    extra_report_dict_2 = {}
    with open('virus_extra_reports2.csv', 'r') as csvfile:
        report2_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in report2_reader:
            extra_report_dict_2[row[0]] = row[1]

    submeta2 = meta[:][7090:]

    submeta2['Report'] = submeta2['Unnamed: 0'].apply(lambda x: extra_report_dict_2[str(x)])

    submeta2.tail()

    meta.to_csv('full_virus_meta.csv', index = False)

In [12]:
# read in the full meta df with all reports.
chunksize = 1000
TextFileReader = pd.read_csv('full_virus_meta.csv', chunksize=chunksize, iterator=True)
meta = pd.concat(TextFileReader, ignore_index=True)

meta.tail()

,Unnamed: 0,Name,BeautifulSoup,ItemId,Report
9456,9456,Wilkie partiti-like virus 1,"<?xml version=""1.0"" encoding=""utf-8""?>\n<!DOCT...",NC_035122.1,LOCUS NC_035122 2100 bp ...
9457,9457,Wilkie partiti-like virus 2,"<?xml version=""1.0"" encoding=""utf-8""?>\n<!DOCT...",NC_035119.1,LOCUS NC_035119 1819 bp ...
9458,9458,Wolkberg virus segment 1,"<?xml version=""1.0"" encoding=""utf-8""?>\n<!DOCT...",NC_034633.1,LOCUS NC_034633 6873 bp ...
9459,9459,Wolkberg virus segment 2,"<?xml version=""1.0"" encoding=""utf-8""?>\n<!DOCT...",NC_034631.1,LOCUS NC_034631 4461 bp ...
9460,9460,Wolkberg virus segment 3,"<?xml version=""1.0"" encoding=""utf-8""?>\n<!DOCT...",NC_034632.1,LOCUS NC_034632 978 bp ...


In [13]:
# Extract genome sequences from the reports, add them to viral_genomes
viral_genomes['Sequence'] = 'None'

def get_genome(report):
    try:
        report_origin = re.findall('ORIGIN[0-9,a-z, ,\n]+', report)
        report_origin = report_origin[0]
        origin = re.findall('[atcg]+', report_origin)
        s = ''
        genome = s.join(origin)
        return(genome)
    except IndexError:
        print('error at', report)
        pass


viral_genomes['Sequence'] = meta['Report'].apply(get_genome)

error at LOCUS       NC_016763              43221 bp    DNA     circular PHG 08-DEC-2014
DEFINITION  Salmonella phage SE2, complete genome.
ACCESSION   NC_016763
VERSION     NC_016763.1
DBLINK      BioProject: PRJNA82643
KEYWORDS    RefSeq.
SOURCE      Salmonella phage SE2
  ORGANISM  Salmonella phage SE2
            Viruses; dsDNA viruses, no RNA stage; Caudovirales; Siphoviridae;
            Guernseyvirinae; Jerseyvirus.
REFERENCE   1  (bases 1 to 43221)
  AUTHORS   Tiwari,B.R., Kim,S. and Kim,J.
  TITLE     Complete Genomic Sequence of Salmonella enterica Serovar
            Enteritidis Phage SE2
  JOURNAL   J. Virol. 86 (14), 7712 (2012)
   PUBMED   22733878
REFERENCE   2  (bases 1 to 43221)
  CONSRTM   NCBI Genome Project
  TITLE     Direct Submission
  JOURNAL   Submitted (10-FEB-2012) National Center for Biotechnology
            Information, NIH, Bethesda, MD 20894, USA
REFERENCE   3  (bases 1 to 43221)
  AUTHORS   Tiwari,B.R., Kim,S. and Kim,J.
  TITLE     Direct Submission
  

In [14]:
viral_genomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9458 entries, 0 to 9457
Data columns (total 13 columns):
Genome                9458 non-null object
Accession             9458 non-null object
Source information    9458 non-null object
Segm                  9458 non-null object
Length                9458 non-null object
Protein               9458 non-null object
Neighbors             9458 non-null object
Host                  9458 non-null object
Created               9458 non-null object
Updated               9458 non-null object
Classification        9458 non-null object
Extra Info            2996 non-null object
Sequence              9457 non-null object
dtypes: object(13)
memory usage: 960.7+ KB


In [15]:
sal_phage = viral_genomes.loc[viral_genomes['Sequence'].isnull()]

In [16]:
##### for some reason the Salmonella phage SE2, complete genome report loaded in without a genome, so I'll try getting that again. 
Entrez.email = 'qdupupet@umass.edu'
handle = Entrez.efetch(db="nuccore", id='NC_016763.1', rettype="gb", retmode="text")
sal_phage_2_report = handle.read()

report_origin = re.findall('ORIGIN[0-9,a-z, ,\n]+', sal_phage_2_report)
report_origin = report_origin[0]
origin = re.findall('[atcg]+', report_origin)
s = ''
sal_phage_2_genome = s.join(origin)



sal_phage['Sequence'] = sal_phage['Sequence'].apply(lambda x: sal_phage_2_genome)

viral_genomes[2879:2880] = sal_phage

In [19]:
viral_genomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9458 entries, 0 to 9457
Data columns (total 13 columns):
Genome                9458 non-null object
Accession             9458 non-null object
Source information    9458 non-null object
Segm                  9458 non-null object
Length                9458 non-null object
Protein               9458 non-null object
Neighbors             9458 non-null object
Host                  9458 non-null object
Created               9458 non-null object
Updated               9458 non-null object
Classification        9458 non-null object
Extra Info            2996 non-null object
Sequence              9458 non-null object
dtypes: object(13)
memory usage: 960.7+ KB


In [20]:
# Extract the molecule type (RNA or DNA) from the report
viral_genomes['Molecule Type'] = 'None'

def extract_genome_type(report):
    try:
        report_mol_type = re.findall('mol_type=[A-Z,a-z,=," ]+', report)
        report_mol_type = report_mol_type[0]
        molecule_type = re.findall('"[A-z ]+"', report_mol_type)
        molecule_type = molecule_type[0].strip('"')
        return(molecule_type)
    except IndexError:
        print('error at', report)
        pass
    
viral_genomes['Molecule Type'] = meta['Report'].apply(extract_genome_type)

In [21]:
# Extract the phylogeny from the report
viral_genomes['Phylogeny'] = 'None'

def extract_phylogeny(report):
    try:
        report_phylo = re.findall('Viruses[A-z ;,]+', report)
        report_phylo = report_phylo[0]
        return(report_phylo)
    except IndexError:
        print('error at', report)
        pass
    
viral_genomes['Phylogeny'] = meta['Report'].apply(extract_phylogeny)

In [22]:
viral_genomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9458 entries, 0 to 9457
Data columns (total 15 columns):
Genome                9458 non-null object
Accession             9458 non-null object
Source information    9458 non-null object
Segm                  9458 non-null object
Length                9458 non-null object
Protein               9458 non-null object
Neighbors             9458 non-null object
Host                  9458 non-null object
Created               9458 non-null object
Updated               9458 non-null object
Classification        9458 non-null object
Extra Info            2996 non-null object
Sequence              9458 non-null object
Molecule Type         9458 non-null object
Phylogeny             9458 non-null object
dtypes: object(15)
memory usage: 1.1+ MB


In [23]:
viral_genomes.to_csv('viral_genomes.csv', index = False)